## Here I will take the key features from my data set and run them through weighted classificaton models, current models are Random Forest and Logit

due to class imbalance and non-linear features my models are currently correctly identifying approx 10- 50% of the failed hard drives and mis attributing approx 5 - 25% of the working drives as broken. I will implement non-linear feature treatment and svm to address this in the coming days.

In [10]:
import pandas as pd
import sklearn
import seaborn as sns

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
from sklearn import svm, ensemble, linear_model, neighbors, naive_bayes

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score

In [14]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, 
                              AdaBoostClassifier, BaggingRegressor)
from sklearn import linear_model, svm, metrics, ensemble

In [15]:
from sklearn import model_selection, preprocessing

In [16]:
df = pd.read_pickle("/Users/robjohns/Data/Metis/HDfailure/Data/tot_2weeks_pickle.pkl")

In [17]:
df.failure.value_counts()

0    31252
1     1486
Name: failure, dtype: int64

In [18]:
df.head()

,serial_number,failure,capacity_bytes,smart_1_normalized,smart_3_normalized,smart_4_raw,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,...,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_raw,smart_198_raw,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
0,S3000FZ5,0,56011018420224,1622.0,1330.0,70.0,0.0,1176.0,4.216763e+09,1304.0,...,1372.0,60914.0,415.0,415.0,0.0,0.0,0.0,1.894106e+15,1.651018e+11,2.639205e+11
1,S3000QAP,0,56011018420224,1615.0,1274.0,112.0,0.0,1176.0,4.558123e+09,1302.0,...,1204.0,401044.0,392.0,392.0,0.0,0.0,0.0,8.732000e+04,2.346867e+11,1.705040e+12
2,S3002HH0,0,56011018420224,1624.0,1288.0,154.0,0.0,1148.0,2.588138e+09,1358.0,...,1302.0,207018.0,345.0,345.0,0.0,0.0,0.0,4.458400e+04,1.399439e+11,5.466639e+11
3,S30034E6,0,56011018420224,1624.0,1330.0,70.0,0.0,1192.0,5.543299e+09,1316.0,...,1386.0,51296.0,318.0,318.0,0.0,0.0,0.0,2.353376e+15,1.693085e+11,3.589584e+11
4,S3003GAB,0,56011018420224,1604.0,1288.0,118.0,0.0,1171.0,3.301111e+09,1344.0,...,1316.0,185027.0,287.0,287.0,0.0,0.0,0.0,5.756200e+04,1.594639e+11,7.925754e+11


In [19]:
df.count()

serial_number           32738
failure                 32738
capacity_bytes          32738
smart_1_normalized      32738
smart_3_normalized      32738
smart_4_raw             32738
smart_5_raw             32738
smart_7_normalized      32738
smart_7_raw             32738
smart_9_normalized      32738
smart_9_raw             32738
smart_12_raw            32738
smart_183_raw           32738
smart_184_raw           32738
smart_187_normalized    32738
smart_187_raw           32738
smart_189_normalized    32738
smart_189_raw           32738
smart_190_raw           32738
smart_192_raw           32738
smart_193_normalized    32738
smart_193_raw           32738
smart_194_normalized    32738
smart_194_raw           32738
smart_197_raw           32738
smart_198_raw           32738
smart_199_raw           32738
smart_240_raw           32738
smart_241_raw           32738
smart_242_raw           32738
dtype: int64

In [34]:
y,X= df['failure'],df.drop(['failure','serial_number','capacity_bytes'],axis=1)

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.2, stratify=y)

In [22]:
x_test.shape

(6548, 27)

In [23]:
#scaler = preprocessing.MinMaxScaler()
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)

In [24]:
print(len(x_train), len(x_test))

26190 6548


In [25]:
#X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=13)

#X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [26]:
def print_metrics(model):
  
    y_pred = model.predict(x_test)

    print(metrics.classification_report(y_test, y_pred))

In [27]:
rf_weights  = {
      0: 1,
      1: 100}
RandomForest_weights = RandomForestClassifier(class_weight=rf_weights,n_estimators=500)
RandomForest_weights.fit(x_train, y_train)

print_metrics(RandomForest_weights)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6251
           1       0.86      0.47      0.61       297

    accuracy                           0.97      6548
   macro avg       0.92      0.73      0.80      6548
weighted avg       0.97      0.97      0.97      6548



In [35]:
rf_weights  = {
      0: 1,
      1: 200}
randomforest = RandomForestClassifier(class_weight=rf_weights,n_estimators=400)

In [36]:
randomforest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 200},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=400, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [37]:
randomforest.score(x_train,y_train)

1.0

In [38]:
rf_conf=confusion_matrix(y_test, randomforest.predict(x_test))
rf_conf


array([[6224,   27],
       [ 163,  134]])

In [39]:
failperc=100*(rf_conf[0])[1]/((rf_conf[0])[0]+(rf_conf[0])[1])
print('RF Incorrectly thinks', failperc,'% are going to fail')

RF Incorrectly thinks 0.4319308910574308 % are going to fail


In [40]:
caught= 100*(rf_conf[1])[1]/((rf_conf[1])[0]+(rf_conf[1])[1])
print('RF Correctly thinks', caught,'% are going to fail')

RF Correctly thinks 45.11784511784512 % are going to fail
